# Mount com o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Configuracao do Spark

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:2.1.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [ ]:
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [ ]:
# Bibliotecas pyspark
from pyspark.sql import SparkSession,SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Define parameters

In [ ]:
#entrada
path_pib =  f'/content/drive/MyDrive/Case_Mercado_Livre/PIB/API_NY.GDP.MKTP.KD.ZG_DS2_es_csv_v2_5358720.csv'
path_tax_juros =  f'/content/drive/MyDrive/Case_Mercado_Livre/Taxa de Juros Ativa/API_FR.INR.LEND_DS2_es_csv_v2_5381469.csv'
path_dados_socioeconomicos =  f'/content/drive/MyDrive/Case_Mercado_Livre/dados socioeconomicos/API_Download_DS2_en_csv_v2_5389446.csv'
path_nav_marketplace_arg =  f'/content/drive/MyDrive/Case_Mercado_Livre/navegacao_marketplaces.csv'
path_desempenho_ml_arg =  f'/content/drive/MyDrive/Case_Mercado_Livre/desempenho-ml-argentina-2022-03.csv'
path_desempenho_ml_br =  f'/content/drive/MyDrive/Case_Mercado_Livre/desempenho-ml-brasil-2022-03.csv'

# saida
path_dest_macroeconomicos = f'/content/drive/MyDrive/Case_Mercado_Livre/output/macroeconomicos'
path_dest_desempenho_ml = f'/content/drive/MyDrive/Case_Mercado_Livre/output/desempenho_ml'
path_dest_nav_marketplace_arg = f'/content/drive/MyDrive/Case_Mercado_Livre/output/nav_marketplace_arg'


# Carregamento das bases para dataframes

In [ ]:
df_pib= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_pib)
df_tax_juros= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_tax_juros)
df_dados_socioeconomicos= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_dados_socioeconomicos)
df_nav_marketplace_arg= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_nav_marketplace_arg)
df_desempenho_ml_arg= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_desempenho_ml_arg)
df_desempenho_ml_br= spark.read.format("csv").option("delimiter", ",").option("header","true").option("encoding", "UTF-8").load(path_desempenho_ml_br)

# selecao de colunas de interesse

In [ ]:
df_pib= df_pib.select("Country Name","Indicator Name","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021")
df_tax_juros= df_tax_juros.select("Country Name","Indicator Name","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021")
df_dados_socioeconomicos= df_dados_socioeconomicos.select("Country Name","Indicator Name","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021")
df_desempenho_ml_arg = df_desempenho_ml_arg.withColumn("pais",lit("Argentina"))
df_desempenho_ml_br = df_desempenho_ml_br.withColumn("pais",lit("Brasil"))

# Filtro das linhas de interesse

In [ ]:
df_pib= df_pib.filter((col("Country Name") == 'Argentina')|(col("Country Name") == 'Brasil'))
df_tax_juros= df_tax_juros.filter((col("Country Name") == 'Argentina')|(col("Country Name") == 'Brasil'))
df_dados_socioeconomicos= df_dados_socioeconomicos.filter((col("Indicator Name") == 'Age dependency ratio (% of working-age population)')\
                                                                  |(col("Indicator Name") == 'Unemployment, total (% of total labor force) (modeled ILO estimate)')\
                                                                  |(col("Indicator Name") == 'GDP per capita (current US$)')\
                                                                  |(col("Indicator Name") == 'Inflation, GDP deflator: linked series (annual %)')\
                                                                  |(col("Indicator Name") == 'Final consumption expenditure (current US$)')\
                                                                  |(col("Indicator Name") == 'Government expenditure per student, secondary (% of GDP per capita)')\
                                                                  |(col("Indicator Name") == 'Mobile cellular subscriptions')\
                                                                  |(col("Indicator Name") == 'Individuals using the Internet (% of population)')\
                                                                  |(col("Indicator Name") == 'New businesses registered (number)')\
                                                                  |(col("Indicator Name") == 'Population, total'))

# Unpivot dos dados

In [ ]:
# PIB
df_pib_pivot = df_pib.withColumn("dados",array(array(col("2011"),lit('2011')),array(col("2012"),lit('2012')),array(col("2013"),lit('2013')),array(col("2014"),lit('2014')),array(col("2015"),lit('2015'))\
                                               ,array(col("2016"),lit('2016')),array(col("2017"),lit('2017')),array(col("2018"),lit('2018')),array(col("2019"),lit('2019')),array(col("2020"),lit('2020'))\
                                               ,array(col("2021"),lit('2021'))))

df_pib_pivot = df_pib_pivot.withColumn("dados",explode("dados"))
df_pib_exp = df_pib_pivot.withColumn("ano", df_pib_pivot["dados"].getItem(1)).withColumn("valor", df_pib_pivot["dados"].getItem(0))
df_pib_final = df_pib_exp.select("Country Name","Indicator Name","ano","valor")

# Taxa de Juros
df_tax_juros_pivot = df_tax_juros.withColumn("dados",array(array(col("2011"),lit('2011')),array(col("2012"),lit('2012')),array(col("2013"),lit('2013')),array(col("2014"),lit('2014')),array(col("2015"),lit('2015'))\
                                               ,array(col("2016"),lit('2016')),array(col("2017"),lit('2017')),array(col("2018"),lit('2018')),array(col("2019"),lit('2019')),array(col("2020"),lit('2020'))\
                                               ,array(col("2021"),lit('2021'))))

df_tax_juros_pivot = df_tax_juros_pivot.withColumn("dados",explode("dados"))
df_tax_juros_exp = df_tax_juros_pivot.withColumn("ano", df_tax_juros_pivot["dados"].getItem(1)).withColumn("valor", df_tax_juros_pivot["dados"].getItem(0))
df_tax_juros_final = df_tax_juros_exp.select("Country Name","Indicator Name","ano","valor")

# dados_socioeconomicos
df_dados_socioeconomicos_pivot = df_dados_socioeconomicos.withColumn("dados",array(array(col("2011"),lit('2011')),array(col("2012"),lit('2012')),array(col("2013"),lit('2013')),array(col("2014"),lit('2014')),array(col("2015"),lit('2015'))\
                                               ,array(col("2016"),lit('2016')),array(col("2017"),lit('2017')),array(col("2018"),lit('2018')),array(col("2019"),lit('2019')),array(col("2020"),lit('2020'))\
                                               ,array(col("2021"),lit('2021'))))

df_dados_socioeconomicos_pivot = df_dados_socioeconomicos_pivot.withColumn("dados",explode("dados"))
df_dados_socioeconomicos_exp = df_dados_socioeconomicos_pivot.withColumn("ano", df_dados_socioeconomicos_pivot["dados"].getItem(1)).withColumn("valor", df_dados_socioeconomicos_pivot["dados"].getItem(0))
df_dados_socioeconomicos_final = df_dados_socioeconomicos_exp.select("Country Name","Indicator Name","ano","valor")

# Uniao dos dados

In [ ]:
df_macroeconomicos = df_pib_final.union(df_tax_juros_final)\
                                  .union(df_dados_socioeconomicos_final)

In [ ]:
df_desempenho_ml = df_desempenho_ml_arg.union(df_desempenho_ml_br)

# calculo da porcentagem da populacao

In [ ]:
df_macroeconomicos = df_macroeconomicos.withColumn("valor",when(col("indicator_name").contains("%"),col("valor")/10000000).otherwise(col("value")))

AnalysisException: ignored

# removendo registros em branco



In [ ]:
df_nav_marketplace_arg = df_nav_marketplace_arg.dropna()

# Renomeando Colunas

In [ ]:
df_macroeconomicos = df_macroeconomicos.withColumnRenamed("Country Name","country_name")\
                                        .withColumnRenamed("Indicator Name","indicator_name")

# display dos bancos de dados para verificacao

In [ ]:
df_pib.toPandas()

,Country Name,Indicator Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Argentina,Crecimiento del PIB (% anual),6.00395169280579,-1.0264204544321,2.40532378079436,-2.51261532081394,2.73115982828944,-2.08032784377811,2.81850297775918,-2.61739646282038,-2.00086100285785,-9.94323513446069,10.3982494646904
1,Brasil,Crecimiento del PIB (% anual),3.97442307944702,1.92117598576537,3.00482266944432,0.503955740242247,-3.54576339269425,-3.27591690782192,1.32286905404399,1.783666761634,1.22077782360842,-3.87867633397198,4.61942162067312


In [ ]:
df_pib_final.toPandas()

,Country Name,Indicator Name,ano,valor
0,Argentina,Crecimiento del PIB (% anual),2011,6.00395169280579
1,Argentina,Crecimiento del PIB (% anual),2012,-1.0264204544321
2,Argentina,Crecimiento del PIB (% anual),2013,2.40532378079436
3,Argentina,Crecimiento del PIB (% anual),2014,-2.51261532081394
4,Argentina,Crecimiento del PIB (% anual),2015,2.73115982828944
5,Argentina,Crecimiento del PIB (% anual),2016,-2.08032784377811
6,Argentina,Crecimiento del PIB (% anual),2017,2.81850297775918
7,Argentina,Crecimiento del PIB (% anual),2018,-2.61739646282038
8,Argentina,Crecimiento del PIB (% anual),2019,-2.00086100285785
9,Argentina,Crecimiento del PIB (% anual),2020,-9.94323513446069


In [ ]:
df_tax_juros.toPandas()

,Country Name,Indicator Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Argentina,Tasa de interés activa (%),14.0875,14.0608333333333,17.1491666666667,24.0091666666667,24.9158333333333,31.2325,26.5808333333333,48.5166666666667,67.2541666666667,29.3875,35.5583333333333
1,Brasil,Tasa de interés activa (%),43.8833333333333,36.6366666666667,27.3916666666667,32.0083333333333,43.9583333333333,52.1,46.9166666666667,39.0833333333333,37.475,29.0416666666667,30.0166666666667


In [ ]:
df_tax_juros_final.toPandas()

,Country Name,Indicator Name,ano,valor
0,Argentina,Tasa de interés activa (%),2011,14.0875
1,Argentina,Tasa de interés activa (%),2012,14.0608333333333
2,Argentina,Tasa de interés activa (%),2013,17.1491666666667
3,Argentina,Tasa de interés activa (%),2014,24.0091666666667
4,Argentina,Tasa de interés activa (%),2015,24.9158333333333
5,Argentina,Tasa de interés activa (%),2016,31.2325
6,Argentina,Tasa de interés activa (%),2017,26.5808333333333
7,Argentina,Tasa de interés activa (%),2018,48.5166666666667
8,Argentina,Tasa de interés activa (%),2019,67.2541666666667
9,Argentina,Tasa de interés activa (%),2020,29.3875


In [ ]:
df_dados_socioeconomicos.toPandas()

,Country Name,Indicator Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Argentina,"Population, total",41261490,41733271,42202935,42669500,43131966,43590368,44044811,44494502,44938712,45376763,45808747
1,Argentina,"Government expenditure per student, secondary ...",20.64525,20.75691,21.62494,21.31954,22.87436,21.59171,20.6226,None,None,None,None
2,Argentina,Age dependency ratio (% of working-age populat...,56.3546668780625,56.1922783212828,56.0868723571096,55.9732762411572,55.8644592427226,55.7543621214315,55.6112391046053,55.4071736223346,55.1353088624017,54.7600781645305,54.2701483886344
3,Argentina,"Unemployment, total (% of total labor force) (...",7.17999982833862,7.21999979019165,7.09999990463257,7.26999998092651,7.52400016784668,8.10900020599365,8.35000038146973,9.22000026702881,9.84000015258789,11.460000038147,8.73999977111816
4,Argentina,GDP per capita (current US$),12848.8641969705,13082.664325572,13080.2547323367,12334.7982453893,13789.060424772,12790.2424732447,14613.041824658,11795.1593866287,9963.6725062053,8496.42414176374,10636.1201956183
5,Argentina,"Inflation, GDP deflator: linked series (annual %)",23.7034721839593,22.3148806733049,23.9487984162746,40.2829716315506,26.5799916024707,41.1193799882279,26.0063792819079,42.0336689575031,49.1955790619497,40.0768830276961,54.1524119312346
6,Argentina,Final consumption expenditure (current US$),423649649448.919,445242277160.302,457042608919.298,433334542562.24,499614480942.488,464849978689.813,543457307147.989,447541502349.172,369714687751.698,311065684471.896,373885266948.238
7,Argentina,Individuals using the Internet (% of population),51,55.8,59.9,64.7,68.04306411,70.96898082,74.29490687,77.7,79.9,85.5,87.14829302
8,Argentina,Mobile cellular subscriptions,60722729,64327647,67361515,61234216,61842011,63719805,61897379,58598041,56352947,54763900,59065827
9,Argentina,New businesses registered (number),5828,4928,5406,4699,4958,5885,6209,5667,None,None,None


In [ ]:
df_dados_socioeconomicos_final.toPandas()

,Country Name,Indicator Name,ano,valor
0,Argentina,"Population, total",2011,41261490
1,Argentina,"Population, total",2012,41733271
2,Argentina,"Population, total",2013,42202935
3,Argentina,"Population, total",2014,42669500
4,Argentina,"Population, total",2015,43131966
...,...,...,...,...
215,Brazil,New businesses registered (number),2017,235300
216,Brazil,New businesses registered (number),2018,253902
217,Brazil,New businesses registered (number),2019,291392
218,Brazil,New businesses registered (number),2020,405049


In [ ]:
df_nav_marketplace_arg.toPandas()

,country,site,Traffic Share,MoM traffic Change,Country Rank,Monthly Visits,Unique Visitors,Visit Duration,Pages,Bounce Rate
0,EUA,amazon.com,13.91%,9.33%,#13,2.359B,448.7M,0:07:09,9.11,35.15%
1,EUA,ebay.com,4.32%,11.83%,#38,731.9M,189.2M,0:06:59,6.85,37.51%
2,Japan,amazon.co.jp,3.37%,7.07%,#54,571.4M,77.27M,0:06:44,7.64,36.16%
3,Japan,rakuten.co.jp,3.05%,9.03%,#56,516.7M,70.40M,0:06:24,7.50,38.09%
4,EUA,etsy.com,2.86%,8.64%,#59,484.2M,188.0M,0:05:46,6.39,46.33%
5,Brazil,aliexpress.com,2.75%,14.04%,#71,466.3M,168.9M,0:05:57,4.83,40.14%
6,Germany,amazon.de,2.65%,9.95%,#57,448.8M,77.14M,0:07:02,9.31,32.61%
7,EUA,walmart.com,2.27%,10.01%,#84,385.4M,139.3M,0:04:38,4.88,52.75%
8,Russia,wildberries.ru,2.16%,12.07%,#68,365.6M,57.74M,0:11:51,12.34,27.02%
9,UK,amazon.co.uk,2.15%,11.03%,#75,364.2M,73.19M,0:06:16,7.80,36.15%


In [ ]:
df_desempenho_ml_arg.toPandas()

,variavel,subvariavel,valor
0,device,mobile Web,51.43%
1,device,Desktop,48.57%
2,num visita,oct/22,137900000
3,num visita,Nov-22,136900000
4,num visita,dec/22,143500000
5,num visita,Jan-23,154600000
6,num visita,Feb-23,131100000
7,num visita,Mar-23,135400000
8,country,Argentina,94.06%
9,country,United States,0.71%


In [ ]:
df_desempenho_ml_br.toPandas()

,variavel,subvariavel,valor
0,device,mobile Web,59.01%
1,device,Desktop,40.99%
2,num visita,oct/22,234100000
3,num visita,Nov-22,244100000
4,num visita,dec/22,256700000
5,num visita,Jan-23,261500000
6,num visita,Feb-23,22400000
7,num visita,Mar-23,24800000
8,country,Brazil,97.39%
9,country,United States,0.70%


In [ ]:
df_macroeconomicos.toPandas()

,country_name,indicator_name,ano,valor
0,Argentina,Crecimiento del PIB (% anual),2011,6.00395169280579
1,Argentina,Crecimiento del PIB (% anual),2012,-1.0264204544321
2,Argentina,Crecimiento del PIB (% anual),2013,2.40532378079436
3,Argentina,Crecimiento del PIB (% anual),2014,-2.51261532081394
4,Argentina,Crecimiento del PIB (% anual),2015,2.73115982828944
...,...,...,...,...
259,Brazil,New businesses registered (number),2017,235300
260,Brazil,New businesses registered (number),2018,253902
261,Brazil,New businesses registered (number),2019,291392
262,Brazil,New businesses registered (number),2020,405049


In [ ]:
df_desempenho_ml.toPandas()

,variavel,subvariavel,valor,pais
0,device,mobile Web,51.43%,Argentina
1,device,Desktop,48.57%,Argentina
2,num visita,oct/22,137900000,Argentina
3,num visita,Nov-22,136900000,Argentina
4,num visita,dec/22,143500000,Argentina
...,...,...,...,...
89,Top Link Destinations,mercadopago.com.ar,7.17%,Brasil
90,Top Link Destinations,mlstatic.com,7.24%,Brasil
91,Top Link Destinations,google.com,2.47%,Brasil
92,Top Link Destinations,starplus.com,2.11%,Brasil


# Escrevendo os dataframes de saida

In [ ]:
df_macroeconomicos\
.coalesce(1)\
.write\
.format("csv")\
.mode("overwrite")\
.save(path_dest_macroeconomicos)

In [ ]:
df_desempenho_ml\
.coalesce(1)\
.write\
.format("csv")\
.mode("overwrite")\
.save(path_dest_desempenho_ml)

In [ ]:
df_nav_marketplace_arg\
.coalesce(1)\
.write\
.format("csv")\
.mode("overwrite")\
.save(path_dest_nav_marketplace_arg)